# Transfer-Learning , Scaled Up models 

### Beating 50.76% Accuracy , accross all classes 
#### Mini Models : 101 classes of data with 10% input 


In [8]:
import tensorflow as tf
import keras
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
print("CUDA Support:", tf.test.is_built_with_cuda())
print("GPU Available:", tf.config.list_physical_devices('GPU'))


CUDA Support: True
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


##### Import Helper Functions 

In [1]:
from helper import *

2025-02-22 10:31:46.791407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740216706.856692  502346 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740216706.874690  502346 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 10:31:46.999909: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 101 Classes of data 
train_dir = "101_food_classes_10_percent/101_food_classes_10_percent/train"
test_dir = "101_food_classes_10_percent/101_food_classes_10_percent/test"

In [3]:
walk_through_dir("/home/akhil/Documents/Tf_Local/101_food_classes_10_percent")

There are 2 directories and 0 images in '/home/akhil/Documents/Tf_Local/101_food_classes_10_percent'.
There are 1 directories and 1 images in '/home/akhil/Documents/Tf_Local/101_food_classes_10_percent/__MACOSX'.
There are 1 directories and 2 images in '/home/akhil/Documents/Tf_Local/101_food_classes_10_percent/__MACOSX/101_food_classes_10_percent'.
There are 0 directories and 34 images in '/home/akhil/Documents/Tf_Local/101_food_classes_10_percent/__MACOSX/101_food_classes_10_percent/test'.
There are 2 directories and 0 images in '/home/akhil/Documents/Tf_Local/101_food_classes_10_percent/101_food_classes_10_percent'.
There are 101 directories and 0 images in '/home/akhil/Documents/Tf_Local/101_food_classes_10_percent/101_food_classes_10_percent/test'.
There are 0 directories and 250 images in '/home/akhil/Documents/Tf_Local/101_food_classes_10_percent/101_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '/home/akhil/Documents/Tf_Local/101_food_classes_10

In [4]:
# Set up data input

import tensorflow as tf
import tf_keras
IMG_SIZE = (224 ,224)
train_data_all_10_percent = tf_keras.preprocessing.image_dataset_from_directory(train_dir ,
                                                                                label_mode = "categorical",
                                                                                image_size = IMG_SIZE)
test_data = tf_keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode = "categorical",
                                                                image_size = IMG_SIZE,
                                                                shuffle = False ) # Dont Shuffle the  test daata for prediction analysis 



Found 7575 files belonging to 101 classes.


I0000 00:00:1740216711.999757  502346 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4024 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Found 25250 files belonging to 101 classes.


- Model Checkpoit 
- Data augmentation layer 
- Build headless fnctional efficientB0 model 
- Compile 
- Feeature Extract for 5 full passes on training data
- Validate on 15 % of Test data 



In [5]:
# Training teh transfer learning model with 10 percent of the data 

# Create Checkpoint Callback 
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf_keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only= True,
                                                         monitor= "accuracy",
                                                         save_best_only=True)

## Pushing Data Augmentaion on GPU 💷

In [6]:
# Data Augmentation Layer

from tf_keras import layers
from tf_keras.layers.experimental import preprocessing 
from tf_keras.models import Sequential

data_augmentation = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2)  #Rescaling is not required in this case , but required in the resnet models
     
], name = "data_augmentation")


In [10]:
# Headless model , with Functional API 
# Baase Model , freeaze some layers

base_model = tf_keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Model Architechure with trinable top layers

inputs = layers.Input(shape = (224,224,3), name = "input_layer")
x = data_augmentation(inputs)  # Computed on GPU during Training phase 
x = base_model(x, training = False) # Freezed Layers 
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(101, activation = "softmax")(x)
model = tf_keras.Model(inputs, outputs)

# Compile the model

model.compile(loss = "categorical_crossentropy",
              optimizer = tf_keras.optimizers.Adam(),
              metrics = ["accuracy"])

# Fit the model

history_10_percent = model.fit(train_data_all_10_percent,
                               epochs = 5,
                               steps_per_epoch = len(train_data_all_10_percent),
                               validation_data = test_data,
                               validation_steps = int(0.15 * len(test_data)),
                               callbacks = [checkpoint_callback])



Epoch 1/5


I0000 00:00:1740238820.246919  503621 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1740238820.991436  503620 service.cc:148] XLA service 0x7a929082ccd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740238820.991660  503620 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2025-02-22 16:40:21.004717: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1740238821.159345  503620 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


237/237 [==============================] - 56s 203ms/step - loss: 3.3641 - accuracy: 0.2737 - val_loss: 2.4797 - val_accuracy: 0.4494
Epoch 2/5
237/237 [==============================] - 42s 177ms/step - loss: 2.2018 - accuracy: 0.4987 - val_loss: 2.0431 - val_accuracy: 0.5042
Epoch 3/5
237/237 [==============================] - 42s 179ms/step - loss: 1.8292 - accuracy: 0.5659 - val_loss: 1.8944 - val_accuracy: 0.5252
Epoch 4/5
237/237 [==============================] - 44s 184ms/step - loss: 1.5878 - accuracy: 0.6182 - val_loss: 1.8127 - val_accuracy: 0.5294
Epoch 5/5
237/237 [==============================] - 44s 187ms/step - loss: 1.4506 - accuracy: 0.6445 - val_loss: 1.7496 - val_accuracy: 0.5482


In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 101)               129381    
                                                             